# Note
PATSTAT DATA is not public. If you want to use, please purchase your own data at https://www.epo.org/searching-for-patents/business/patstat.html

In [ ]:
%matplotlib inline  
import glob
import zipfile
import gc
import csv
import pandas as pd
import matplotlib.pyplot as plt

datadir = "" # Directories of PATSTAT Data

def gini(list_of_values):
    if(list_of_values.count(list_of_values[0]) == len(list_of_values)):
        if(list_of_values[0] == 0):
            return -1
        else:
            return 0
    sorted_list = sorted(list_of_values)
    height, area = 0, 0
    for value in sorted_list:
        height += value
        area += height - value / 2.
    fair_area = height * len(list_of_values) / 2
    return (fair_area - area) / fair_area


In [ ]:
tls201_files = glob.glob(datadir + "tls201*.zip")
print tls201_files
tls201 = pd.concat((pd.read_csv(f, compression="zip", usecols=['appln_id','appln_auth','appln_nr','appln_kind',
                                                               'appln_filing_year', 'granted']) for f in tls201_files))

In [ ]:
tls207_files = glob.glob(datadir + "tls207*.zip")
tls207 = pd.concat((pd.read_csv(f, compression="zip",  usecols=['person_id', 'appln_id','applt_seq_nr','invt_seq_nr']) for f in tls207_files))

In [ ]:
tls201['appln_auth'] = map(lambda x: str(x).upper(), tls201['appln_auth'])
tls201_count_test_01 = tls201.groupby(['appln_auth']).count()

In [ ]:
del(tls201_count_test_01)

In [ ]:
tls_207_201 = tls207.merge(tls201)

In [ ]:
tls_207_201_FILTERED = (tls_207_201[:][(tls_207_201["appln_kind"] =="A ") & (tls_207_201["appln_filing_year"] >= 2000) 
                                      & (tls_207_201["appln_filing_year"] <= 2017)])


In [ ]:
len(tls_207_201_FILTERED)

In [ ]:
tls_207_201_FILTERED_APPLT_ONLY = (tls_207_201_FILTERED[:][(tls_207_201_FILTERED["applt_seq_nr"] != 0)])
tls_207_201_FILTERED_INVT_ONLY = (tls_207_201_FILTERED[:][(tls_207_201_FILTERED["invt_seq_nr"] != 0)])

In [ ]:
tls_207_201_FILTERED_APPLT_ONLY_AID = tls_207_201_FILTERED_APPLT_ONLY.groupby('appln_auth')['appln_id'].nunique()

In [ ]:
tls_207_201_FILTERED_APPLT_ONLY_PID = tls_207_201_FILTERED_APPLT_ONLY.groupby('appln_auth')['person_id'].nunique()

In [ ]:
tls_207_201_FILTERED_INVT_ONLY_AID = tls_207_201_FILTERED_INVT_ONLY.groupby('appln_auth')['appln_id'].nunique()

In [ ]:
tls_207_201_FILTERED_INVT_ONLY_PID = tls_207_201_FILTERED_INVT_ONLY.groupby('appln_auth')['person_id'].nunique()

In [ ]:
APPLT_COUNT = pd.concat([tls_207_201_FILTERED_APPLT_ONLY_PID, tls_207_201_FILTERED_APPLT_ONLY_AID], axis=1)

In [ ]:
plt.plot(APPLT_COUNT['person_id'], APPLT_COUNT['appln_id'], 'o')
plt.yscale('log')
plt.xscale('log')
plt.xlabel('Number of applicants')
plt.ylabel('Number of patents')

In [ ]:
INVT_COUNT = pd.concat([tls_207_201_FILTERED_INVT_ONLY_PID, tls_207_201_FILTERED_INVT_ONLY_AID], axis=1)

In [ ]:
plt.plot(INVT_COUNT['person_id'], INVT_COUNT['appln_id'], 'o')
plt.yscale('log')
plt.xscale('log')
plt.xlabel('Number of inventors')
plt.ylabel('Number of patents')

In [ ]:
tls_207_201_FILTERED_AID = tls_207_201_FILTERED.groupby('appln_auth')['appln_id'].nunique()
tls_207_201_FILTERED_PID = tls_207_201_FILTERED.groupby('appln_auth')['person_id'].nunique()
ALL_COUNT = pd.concat([tls_207_201_FILTERED_PID, tls_207_201_FILTERED_AID], axis=1)
plt.plot(ALL_COUNT['person_id'], ALL_COUNT['appln_id'], 'o')
plt.yscale('log')
plt.xscale('log')
plt.xlabel('Number of distinct inventors/applicants')
plt.ylabel('Number of patents')

In [ ]:
tls_207_201_FILTERED_APPLT_ONLY_PIDCOUNT = tls_207_201_FILTERED_APPLT_ONLY.groupby(['appln_auth', 'person_id']).count()

In [ ]:
tls_207_201_FILTERED_INVT_ONLY_PIDCOUNT = tls_207_201_FILTERED_INVT_ONLY.groupby(['appln_auth', 'person_id']).count()
tls_207_201_FILTERED_APPLT_ONLY_PIDCOUNT = tls_207_201_FILTERED_APPLT_ONLY.groupby(['appln_auth', 'person_id']).count()

In [ ]:
gc.collect()

In [ ]:
gc.collect()
gini(list(tls_207_201_FILTERED_APPLT_ONLY_PIDCOUNT.loc["KR"]['appln_id'].values))

In [ ]:
countries = tls_207_201_FILTERED_APPLT_ONLY_PIDCOUNT.index.get_level_values(0)
set(countries)
GINI_APPLT = []
for ct in list(set(countries)):
    templist = [ct, gini(list(tls_207_201_FILTERED_APPLT_ONLY_PIDCOUNT.loc[ct]['appln_id'].values))]
    GINI_APPLT.append(templist)
    #print(ct, gini(list(tls_207_201_FILTERED_APPLT_ONLY_PIDCOUNT.loc[ct]['appln_id'].values)))
print GINI_APPLT

In [ ]:
countries = tls_207_201_FILTERED_INVT_ONLY_PIDCOUNT.index.get_level_values(0)
set(countries)
GINI_INVT = []
for ct in list(set(countries)):
    templist = [ct, gini(list(tls_207_201_FILTERED_INVT_ONLY_PIDCOUNT.loc[ct]['appln_id'].values))]
    GINI_INVT.append(templist)
    #print(ct, gini(list(tls_207_201_FILTERED_APPLT_ONLY_PIDCOUNT.loc[ct]['appln_id'].values)))
#print GINI_INVT

In [ ]:
print len(GINI_APPLT)

In [ ]:
print len(GINI_INVT)

In [ ]:
APPLT_COUNT_GINI = APPLT_COUNT.reset_index().merge(
    pd.DataFrame.from_records(GINI_APPLT, columns=['appln_auth','gini_index']), on='appln_auth')
plt.plot(APPLT_COUNT_GINI['appln_id'], APPLT_COUNT_GINI['gini_index'], 'o')
plt.xscale('log')
plt.xlabel('Number of patent')
plt.ylabel('Gini index of Number of Patent per applicant')

In [ ]:
APPLT_COUNT_GINI = APPLT_COUNT.reset_index().merge(
    pd.DataFrame.from_records(GINI_APPLT, columns=['appln_auth','gini_index']), on='appln_auth')
plt.plot(APPLT_COUNT_GINI['person_id'], APPLT_COUNT_GINI['gini_index'], 'o')
plt.xscale('log')
plt.xlabel('Number of applicants')
plt.ylabel('Gini index of Number of Patent per applicant')

In [ ]:
INVT_COUNT_GINI = INVT_COUNT.reset_index().merge(
    pd.DataFrame.from_records(GINI_INVT, columns=['appln_auth','gini_index']), on='appln_auth')
plt.plot(INVT_COUNT_GINI['appln_id'], INVT_COUNT_GINI['gini_index'], 'o')
plt.xscale('log')
plt.xlabel('Number of patent')
plt.ylabel('Gini index of Number of Patent per inventor')

In [ ]:
INVT_COUNT_GINI = INVT_COUNT.reset_index().merge(
    pd.DataFrame.from_records(GINI_INVT, columns=['appln_auth','gini_index']), on='appln_auth')
plt.plot(INVT_COUNT_GINI['person_id'], INVT_COUNT_GINI['gini_index'], 'o')
plt.xscale('log')
plt.xlabel('Number of inventors')
plt.ylabel('Gini index of Number of Patent per inventor')

In [ ]:
APPLT_COUNT_GINI.to_csv("./output/gini_applicant.csv", sep="\t")
INVT_COUNT_GINI.to_csv("./output/gini_inventor.csv", sep="\t" )